In [1]:
from bert import BertModel

/home/krypticmouse/.local/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model = BertModel.from_pretrained('bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertModel were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['bert.encoder.layer.1.attention.output.adapter.proj_down.bias', 'be

In [3]:
from dataset import CoLADataset
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [4]:
data = CoLADataset('raw/in_domain_train.tsv', tokenizer)

In [10]:
model(**data[0]['sentence'], output_hidden_states=True).hidden_states[-1]

tensor([[[ 0.4008,  0.1898,  0.1470,  ..., -0.3015,  0.2269,  0.5410],
         [ 0.5970, -0.0165, -0.3991,  ..., -0.1948,  0.8729,  0.5442],
         [ 0.4604, -0.0840,  0.1409,  ..., -0.4918, -0.1896,  0.1967],
         ...,
         [ 0.5255,  0.3843, -0.0321,  ..., -0.4043,  0.2398,  0.4467],
         [ 0.6178,  0.3638,  0.4818,  ...,  0.2736, -0.7449, -0.4608],
         [ 0.1716,  0.4733,  0.6120,  ..., -0.1511, -0.2494,  0.1768]]],
       grad_fn=<NativeLayerNormBackward0>)

In [13]:
model(**data[0]['sentence']).last_hidden_state[0,0,:].shape

torch.Size([768])

['embeddings.LayerNorm.weight',
 'embeddings.LayerNorm.bias',
 'encoder.layer.0.attention.output.LayerNorm.weight',
 'encoder.layer.0.attention.output.LayerNorm.bias',
 'encoder.layer.0.attention.output.adapter.proj_down.weight',
 'encoder.layer.0.attention.output.adapter.proj_down.bias',
 'encoder.layer.0.attention.output.adapter.proj_up.weight',
 'encoder.layer.0.attention.output.adapter.proj_up.bias',
 'encoder.layer.0.output.LayerNorm.weight',
 'encoder.layer.0.output.LayerNorm.bias',
 'encoder.layer.0.output.adapter.proj_down.weight',
 'encoder.layer.0.output.adapter.proj_down.bias',
 'encoder.layer.0.output.adapter.proj_up.weight',
 'encoder.layer.0.output.adapter.proj_up.bias',
 'encoder.layer.1.attention.output.LayerNorm.weight',
 'encoder.layer.1.attention.output.LayerNorm.bias',
 'encoder.layer.1.attention.output.adapter.proj_down.weight',
 'encoder.layer.1.attention.output.adapter.proj_down.bias',
 'encoder.layer.1.attention.output.adapter.proj_up.weight',
 'encoder.layer.1.

In [20]:
model.named_parameters()

<generator object Module.named_parameters at 0x7f77a9dd9fc0>